In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential # to see the sequence like ip --> hl1 --> hl2--> hl3 -->o/p #to build layer by layer architecture
from keras.layers import Dense # to decide the depth of layers
import numpy as np

/Users/saro/Desktop/dvdrentaldeeplearning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load the dataset
file_path = 'dvd rental final csv.csv'
df = pd.read_csv(file_path)

# Inspect the data
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14596 entries, 0 to 14595
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   inventory_id      14596 non-null  int64  
 1   film_id           14596 non-null  int64  
 2   title             14596 non-null  object 
 3   rental_duration   14596 non-null  int64  
 4   rental_rate       14596 non-null  float64
 5   length            14596 non-null  int64  
 6   replacement_cost  14596 non-null  float64
 7   rating            14596 non-null  object 
 8   category          14596 non-null  object 
 9   language          14596 non-null  object 
 10  store_id          14596 non-null  int64  
 11  rental_id         14596 non-null  int64  
 12  rental_date       14596 non-null  object 
 13  customer_id       14596 non-null  int64  
 14  return_date       14413 non-null  object 
 15  amount            14596 non-null  float64
 16  active            14596 non-null  int64 

In [3]:
df.dropna(inplace=True)

In [4]:
# Convert the 'date' column to datetime
df['rental_date'] = pd.to_datetime(df['rental_date'])

# Extract year, month, day, hour, minute, and second
df['rental_year'] = df['rental_date'].dt.year
df['rental_month'] = df['rental_date'].dt.month
df['rental_day'] = df['rental_date'].dt.day
df['rental_hour'] = df['rental_date'].dt.hour
df['rental_minute'] = df['rental_date'].dt.minute

df.drop('rental_date', axis=1, inplace=True)

df.head()

,inventory_id,film_id,title,rental_duration,rental_rate,length,replacement_cost,rating,category,language,...,rental_id,customer_id,return_date,amount,active,rental_year,rental_month,rental_day,rental_hour,rental_minute
0,3419,749,Rules Human,6,4.99,153,19.99,R,Horror,English,...,1520,341,2005-06-24 23:46:20,7.99,1,2005,6,15,23,57
1,2512,552,Majestic Floats,5,0.99,130,15.99,PG,Documentary,English,...,1778,341,2005-06-22 16:08:48,1.99,1,2005,6,16,18,54
2,2507,551,Maiden Home,3,4.99,138,9.99,PG,New,English,...,1849,341,2005-06-23 18:37:19,7.99,1,2005,6,17,0,13
3,2047,445,Hyde Doctor,5,2.99,100,11.99,G,Classics,English,...,2829,341,2005-06-24 18:10:30,2.99,1,2005,6,19,21,11
4,2569,563,Massacre Usual,6,4.99,165,16.99,R,Games,English,...,3130,341,2005-06-29 18:05:22,7.99,1,2005,6,20,19,3


In [5]:
df['return_date'] = pd.to_datetime(df['return_date'])

# Extract year, month, day, hour, minute, and second
df['return_year'] = df['return_date'].dt.year
df['return_month'] = df['return_date'].dt.month
df['return_day'] = df['return_date'].dt.day
df['return_hour'] = df['return_date'].dt.hour
df['return_minute'] = df['return_date'].dt.minute

df.drop('return_date', axis=1, inplace=True)

In [6]:
df.isnull().sum()

inventory_id        0
film_id             0
title               0
rental_duration     0
rental_rate         0
length              0
replacement_cost    0
rating              0
category            0
language            0
store_id            0
rental_id           0
customer_id         0
amount              0
active              0
rental_year         0
rental_month        0
rental_day          0
rental_hour         0
rental_minute       0
return_year         0
return_month        0
return_day          0
return_hour         0
return_minute       0
dtype: int64

In [7]:
df.drop('language',axis=1, inplace=True)
df.drop('rental_year',axis=1, inplace=True)
df.drop('return_year',axis=1, inplace=True)
df.drop('rental_hour',axis=1, inplace=True)
df.drop('rental_minute',axis=1, inplace=True)
df.drop('return_hour',axis=1, inplace=True)
df.drop('return_minute',axis=1, inplace=True)
df.drop('title',axis=1,inplace=True)
df.drop(['inventory_id','film_id','store_id','rental_id','customer_id'], axis=1, inplace=True)

In [8]:
df.head()

,rental_duration,rental_rate,length,replacement_cost,rating,category,amount,active,rental_month,rental_day,return_month,return_day
0,6,4.99,153,19.99,R,Horror,7.99,1,6,15,6,24
1,5,0.99,130,15.99,PG,Documentary,1.99,1,6,16,6,22
2,3,4.99,138,9.99,PG,New,7.99,1,6,17,6,23
3,5,2.99,100,11.99,G,Classics,2.99,1,6,19,6,24
4,6,4.99,165,16.99,R,Games,7.99,1,6,20,6,29


In [9]:
categorical_column_to_iterate = ['rating', 'category']
continuous_column_to_iterate = ['amount', 'rental_rate', 'replacement_cost']

In [10]:
df_encoded = pd.get_dummies(df,columns=['category','rating'])

In [11]:
df_encoded.columns

Index(['rental_duration', 'rental_rate', 'length', 'replacement_cost',
       'amount', 'active', 'rental_month', 'rental_day', 'return_month',
       'return_day', 'category_Action', 'category_Animation',
       'category_Children', 'category_Classics', 'category_Comedy',
       'category_Documentary', 'category_Drama', 'category_Family',
       'category_Foreign', 'category_Games', 'category_Horror',
       'category_Music', 'category_New', 'category_Sci-Fi', 'category_Sports',
       'category_Travel', 'rating_G', 'rating_NC-17', 'rating_PG',
       'rating_PG-13', 'rating_R'],
      dtype='object')

In [12]:
df_encoded.shape

(14413, 31)

In [13]:
# #encoding
# encoders = {}
# for col in categorical_column_to_iterate:
#     encoder = OneHotEncoder(sparse_output=False)
#     df[col] = encoder.fit_transform(df[[col]])
#     encoders[col] = encoder

# with open('onehotencoder.pkl', 'wb') as f:
#     pickle.dump(encoders, f)

In [14]:
df_encoded.isnull().sum()

rental_duration         0
rental_rate             0
length                  0
replacement_cost        0
amount                  0
active                  0
rental_month            0
rental_day              0
return_month            0
return_day              0
category_Action         0
category_Animation      0
category_Children       0
category_Classics       0
category_Comedy         0
category_Documentary    0
category_Drama          0
category_Family         0
category_Foreign        0
category_Games          0
category_Horror         0
category_Music          0
category_New            0
category_Sci-Fi         0
category_Sports         0
category_Travel         0
rating_G                0
rating_NC-17            0
rating_PG               0
rating_PG-13            0
rating_R                0
dtype: int64

In [15]:
#encoding
encoders_cont = {}
for col in continuous_column_to_iterate:
    encoder = LabelEncoder()
    df_encoded[col] = encoder.fit_transform(df_encoded[col])
    encoders_cont[col] = encoder

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(encoders_cont, f)

In [16]:
encoders_cont

{'amount': LabelEncoder(),
 'rental_rate': LabelEncoder(),
 'replacement_cost': LabelEncoder()}

In [17]:
df_encoded['replacement_cost']

0        10
1         6
2         0
3         2
4         7
         ..
14409     2
14410    20
14411     1
14412     1
14413    12
Name: replacement_cost, Length: 14413, dtype: int64

In [18]:
df_encoded.head()

,rental_duration,rental_rate,length,replacement_cost,amount,active,rental_month,rental_day,return_month,return_day,...,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,6,2,153,10,7,1,6,15,6,24,...,False,False,False,False,False,False,False,False,False,True
1,5,0,130,6,1,1,6,16,6,22,...,False,False,False,False,False,False,False,True,False,False
2,3,2,138,0,7,1,6,17,6,23,...,False,True,False,False,False,False,False,True,False,False
3,5,1,100,2,2,1,6,19,6,24,...,False,False,False,False,False,True,False,False,False,False
4,6,2,165,7,7,1,6,20,6,29,...,False,False,False,False,False,False,False,False,False,True


In [19]:
# Separate features and target
X = df_encoded.drop('amount', axis=1)  # Replace 'revenue_column' with the actual column name for revenue
y = df_encoded['amount']

In [20]:
X.shape, y.shape

((14413, 30), (14413,))

In [21]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape, y_train.shape

((11530, 30), (11530,))

In [23]:
y_train.value_counts()

amount
4     2696
2     2565
0     2108
5      938
6      834
3      804
7      519
1      466
8      338
9      177
10      79
11       6
Name: count, dtype: int64

In [24]:
# Scale the features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [25]:
print(type(X_train))  # Should be <class 'numpy.ndarray'>
print(type(y_train))  # Should be <class 'numpy.ndarray'>
print((X_train.shape))  # Should be <class 'numpy.ndarray'>
print((y_train.shape))  # Should be <class 'numpy.ndarray'>

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
(11530, 30)
(11530,)


In [26]:
# Ensure data is in the correct format
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)


In [27]:
X_train.shape

(11530, 30)

In [28]:
model = Sequential()
# Input layer (match input shape to your data)
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# Hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer for regression (single node, no activation function)
model.add(Dense(1))

# Compile the model for regression with 'mse' loss function
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss (MSE): {test_loss}')

Epoch 1/50


/Users/saro/Desktop/dvdrentaldeeplearning/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 754us/step - loss: 16.9690 - val_loss: 2.2281
Epoch 2/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - loss: 1.5348 - val_loss: 0.7367
Epoch 3/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 574us/step - loss: 0.6764 - val_loss: 0.8169
Epoch 4/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step - loss: 0.5310 - val_loss: 0.5727
Epoch 5/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - loss: 0.3871 - val_loss: 0.7262
Epoch 6/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - loss: 0.3868 - val_loss: 0.2795
Epoch 7/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - loss: 0.2807 - val_loss: 0.4141
Epoch 8/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step - loss: 0.2565 - val_loss: 0.2193
Epoch 9/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 0.2350 - val_loss: 0.1760
Epoch 10/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step - loss: 0.2258 - val_loss: 0.1444
Epoch 11/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 0.1902 - val_loss: 0.1992
Epoch 12/50
361/361 ━━━━━━━━━━━━━━━━━━━━

In [29]:
model.predict(X_test)

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step


array([[5.041713  ],
       [2.0314846 ],
       [0.97167844],
       ...,
       [5.9775553 ],
       [4.0306063 ],
       [3.0407584 ]], dtype=float32)

In [30]:
model.evaluate (x = X_test, y = y_test)

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step - loss: 0.0118


0.01278508547693491

In [31]:
model1 = Sequential()
# Input layer (match input shape to your data)
model1.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# Hidden layers
model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))

# Output layer for regression (single node, no activation function)
model1.add(Dense(1))

# Compile the model for regression with 'mse' loss function
model1.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model1.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# Evaluate the model
test_loss = model1.evaluate(X_test, y_test)
print(f'Test Loss (MSE): {test_loss}')

Epoch 1/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 555us/step - loss: 8.7487 - val_loss: 1.6182
Epoch 2/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - loss: 1.5225 - val_loss: 1.1202
Epoch 3/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step - loss: 0.8792 - val_loss: 0.5345
Epoch 4/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - loss: 0.6623 - val_loss: 0.5895
Epoch 5/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - loss: 0.6004 - val_loss: 0.7603
Epoch 6/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - loss: 0.4702 - val_loss: 0.4238
Epoch 7/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step - loss: 0.3345 - val_loss: 0.3337
Epoch 8/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step - loss: 0.3287 - val_loss: 0.1946
Epoch 9/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step - loss: 0.2791 - val_loss: 0.3147
Epoch 10/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - loss: 0.3020 - val_loss: 0.1722
Epoch 11/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - loss: 0.2189 - val_loss: 0.2258
Epoch 12/100
361/36

In [48]:
# Sigmoid: activation='sigmoid'
# Tanh: activation='tanh'
# ReLU: activation='relu'
# Leaky ReLU: activation=tf.keras.layers.LeakyReLU(alpha=0.01)
# ELU: activation='elu'
# Swish: activation=tf.keras.activations.swish
# Softmax: activation='softmax'
# Softplus: activation='softplus'
# GELU: activation=tf.keras.activations.gelu

In [32]:
model1.save ("firstdl.h5")

In [33]:
d = X_test[0]

In [34]:
d

array([  4.,   1., 155.,  16.,   1.,   8.,  23.,   8.,  30.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.], dtype=float32)

In [35]:
from keras.models import load_model
new_model = load_model("firstdl.h5")
new_model.predict(X_test)

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step


array([[5.2393737],
       [2.084231 ],
       [1.092907 ],
       ...,
       [6.1006265],
       [4.0124335],
       [3.1651633]], dtype=float32)

In [36]:
X.columns

Index(['rental_duration', 'rental_rate', 'length', 'replacement_cost',
       'active', 'rental_month', 'rental_day', 'return_month', 'return_day',
       'category_Action', 'category_Animation', 'category_Children',
       'category_Classics', 'category_Comedy', 'category_Documentary',
       'category_Drama', 'category_Family', 'category_Foreign',
       'category_Games', 'category_Horror', 'category_Music', 'category_New',
       'category_Sci-Fi', 'category_Sports', 'category_Travel', 'rating_G',
       'rating_NC-17', 'rating_PG', 'rating_PG-13', 'rating_R'],
      dtype='object')

In [43]:
d = X_test[0]

In [44]:
d

array([  4.,   1., 155.,  16.,   1.,   8.,  23.,   8.,  30.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.], dtype=float32)

In [45]:
new_model.predict(np.array([d]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


array([[5.2393727]], dtype=float32)